<a href="https://colab.research.google.com/github/Youruler1/Speech-Processing-Lab-Material/blob/main/ASR_Pipeline_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchaudio
from torch.utils.data import DataLoader
from torchaudio.datasets import LIBRISPEECH
from torchaudio.transforms import MelSpectrogram, Resample
import torchaudio.functional as F
import os


train_dataset = LIBRISPEECH("./", url="train-clean-100", download=True)

100%|██████████| 5.95G/5.95G [03:07<00:00, 34.1MB/s]


In [ ]:
import string
CHAR_VOCAB = list(string.ascii_lowercase + " '")  # space and apostrophe included
CHAR2IDX = {c: i + 1 for i, c in enumerate(CHAR_VOCAB)}  # index 0 reserved for padding
IDX2CHAR = {i: c for c, i in CHAR2IDX.items()}
BLANK_IDX = 0


In [ ]:
import torch
import torchaudio
import torchaudio.transforms as T
from torch.nn.utils.rnn import pad_sequence

mel_spec = T.MelSpectrogram(sample_rate=16000, n_fft=400, hop_length=160, n_mels=80)
amplitude_to_db = T.AmplitudeToDB()

def text_to_indices(text):
    return [CHAR2IDX[c] for c in text.lower() if c in CHAR2IDX]

def collate_fn(batch):
    waveforms, labels = [], []
    input_lengths, label_lengths = [], []

    for waveform, _, transcript,_ , _, _ in batch:
        features = amplitude_to_db(mel_spec(waveform)).squeeze(0).transpose(0, 1)  # (time, features)
        waveforms.append(features)
        label_seq = torch.tensor(text_to_indices(transcript), dtype=torch.long)
        labels.append(label_seq)
        input_lengths.append(features.shape[0])
        label_lengths.append(len(label_seq))

    waveforms = pad_sequence(waveforms, batch_first=True)
    labels = pad_sequence(labels, batch_first=True)
    return waveforms, labels, torch.tensor(input_lengths), torch.tensor(label_lengths)


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)

In [ ]:
import torch.nn as nn

class CNNLSTM_ASR(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=3):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(input_dim, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Conv1d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm1d(128),
        )
        self.lstm = nn.LSTM(input_size=128, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # Bidirectional

    def forward(self, x, input_lengths):
        # x shape: (batch, time, features) → CNN expects (batch, features, time)
        x = x.transpose(1, 2)  # (B, F, T)
        x = self.cnn(x)
        x = x.transpose(1, 2)  # (B, T, F)

        # Pack padded sequences for LSTM
        packed = nn.utils.rnn.pack_padded_sequence(x, input_lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_outputs, _ = self.lstm(packed)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs, batch_first=True)
        logits = self.fc(outputs)  # (B, T, Vocab)
        return logits


In [ ]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
VOCAB_SIZE = len(CHAR2IDX) + 1  # +1 for blank token

model = CNNLSTM_ASR(input_dim=80, hidden_dim=256, output_dim=VOCAB_SIZE).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
ctc_loss = nn.CTCLoss(blank=BLANK_IDX, zero_infinity=True)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)

In [ ]:
for epoch in range(10):
    model.train()
    total_loss = 0
    for batch in train_loader:
        inputs, targets, input_lengths, target_lengths = batch
        inputs, targets = inputs.to(device), targets.to(device)

        logits = model(inputs, input_lengths)
        log_probs = nn.functional.log_softmax(logits, dim=-1)
        log_probs = log_probs.transpose(0, 1)  # CTC expects (T, B, C)

        loss = ctc_loss(log_probs, targets, input_lengths, target_lengths)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss = {total_loss / len(train_loader):.4f}")